In [25]:
using Revise

In [4]:
using Turing

In [7]:
using UnPack

In [10]:
using ImmersedLayers

In [29]:
using HeatForecastAnyShape

In [12]:
using CairoMakie
using LaTeXStrings

In [13]:
using Distributions
using Random
using LinearAlgebra
using GaussianMixtures
using Statistics

In [14]:
using JLD2

In [36]:
@model function heat_equation_model(x,t,ystar,Σϵ,obs,gridConfig,bounds)
    config = obs.config
    sens = obs.sens
    Nq = config.Nq
    state_id = config.state_id
    g = gridConfig.g
    cache = gridConfig.cache
    Ntheta = gridConfig.Ntheta

    #Defining priors for the parameters
    x_ids = state_id["heater x"]
    y_ids = state_id["heater y"]
    q_ids = state_id["heater q"]
    c1_ids = state_id["heater c1"]
    c2_ids = state_id["heater c2"]

    #Custom log-uniform prior for the parameters
    xq_prior = log_uniform(x[x_ids],bounds[x_ids])
    yq_prior = log_uniform(x[y_ids], bounds[y_ids])
    qq_prior = log_uniform(x[q_ids], bounds[q_ids])
    c1q_prior = log_uniform(x[c1_ids], bounds[c1_ids])
    c2q_prior = log_uniform(x[c2_ids], bounds[c2_ids])

    #Custom log-uniform prior for the parameters
    xq ~ xq_prior
    yq ~ yq_prior
    qq ~ qq_prior
    c1q ~ c1q_prior
    c2q ~ c2q_prior

    #likelihood
    logp̃ = normal_loglikelihood(x, t, ystar, Σϵ, obs, gridConfig)

    #adding priors and likelihood
    logp̃ += log_uniform_prior(x, bounds)

    return logp̃
end

heat_equation_model (generic function with 2 methods)

### True data

In [27]:
δ_true = 0.01
zq = [0.5+1.2im]
qq = [1.0]
c1q = [0.5]
c2q = [0.25]
Nq_true = length(zq)

1

### Set up the observer for the true data

In [16]:
config_true, x_true = get_config_and_state(zq,qq,c1q,c2q)
ϵmeas_data = [1e-5,1e-4,2.5e-4,3.75e-4,5e-4,6.25e-4,8e-4,1e-3,1.5e-3,2e-3,3e-3,4e-3];
x_true

5-element Vector{Float64}:
 0.5
 1.2
 1.0
 0.5
 0.25

### Heater estimation

In [17]:
Nq_estimator = 1
δ = 0.01
config_estimator = HeaterConfig(Nq_estimator)


# ranges to confine the prior mean to
xr = (-2.0,2.0)
yr = (0.01,2) #(0.01,4) #(0.01,2.0)
qr = (-2,2)
c1r = (0,1)
c2r = (0,1)
bounds = create_state_bounds(xr,yr,qr,c1r,c2r,config_estimator);
bounds

5-element Vector{Tuple{Float64, Float64}}:
 (-2.0, 2.0)
 (0.01, 2.0)
 (-2.0, 2.0)
 (0.0, 1.0)
 (0.0, 1.0)

In [18]:
#bounds for numerical domain
xr = (-2.0,2.0)
yr = (-2.0,2.0) #(0.01,4) #(0.01,2.0)
qr = (-2,2)
c1r = (0,1)
c2r = (0,1)
bounds_num = create_state_bounds(xr,yr,qr,c1r,c2r,config_estimator);
bounds_num

5-element Vector{Tuple{Float64, Float64}}:
 (-2.0, 2.0)
 (-2.0, 2.0)
 (-2.0, 2.0)
 (0.0, 1.0)
 (0.0, 1.0)

### Set up the MCMC parameters

In [19]:
## Parallel tempering ##
propvarX_coarse = 0.01^2 
propvarY_coarse = 0.01^2
propvarQ_coarse = 0.05^2 #0.05^2
propvarC_coarse = 0.01^2

propvarX_fine = 0.005^2 # 0.01^2
propvarY_fine = 0.005^2 # 0.01^2
propvarQ_fine = 0.01^2 # 0.005^2 # 0.01^2
propvarC_fine = 0.005^2

#Ntrial = [10000, 600000]
#propvar = [state_covariance(propvarX_coarse,propvarY_coarse,propvarQ_coarse,propvarR_coarse,config_estimator),
           #state_covariance(propvarX_fine,propvarY_fine,propvarQ_fine,propvarR_fine,config_estimator)];
Ntrial = [10000]
propvar = [state_covariance(propvarX_coarse,propvarY_coarse,propvarQ_coarse,propvarC_coarse,config_estimator)];

### Getting results for a specific case

In [20]:
Δx = 0.01
Ntheta = 500
gridConfig = constructGrids(Δx,bounds_num;Ntheta)

constructGrids{Tuple{Float64, Float64}, BasicILMCache{0, GridScaling, Nodes{Primal, 408, 404, Float64, Matrix{Float64}}, 2, BodyList, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Regularize{0, false}, RegularizationMatrix{Edges{Primal, 408, 404, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Edges{Primal, 408, 404, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, RegularizationMatrix{Nodes{Primal, 408, 404, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Nodes{Primal, 408, 404, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, Nothing, Nothing, Nothing, Nothing, Laplacian{408, 404, Float64, true, false}, Edges{Primal, 408, 404, Float64, Vector{Float64}}, Nodes{Dual, 408, 404, Float64, Matrix{Float64}}, Nothing, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Nothing}}(0.01, (-2.0, 2.0), (-2.0, 2.

In [30]:
K = 1
Nsens = 5
ϵmeas = 5e-4
obs_true, ystar, H, Σϵ, Σx = get_truth_data(Nsens,ϵmeas,x_true,config_true,gridConfig)
logp̃_fcn, obs = setup_estimator(obs_true.sens,bounds,config_estimator,ystar,Σϵ,gridConfig);

In [50]:
bounds

5-element Vector{Tuple{Float64, Float64}}:
 (-2.0, 2.0)
 (0.01, 2.0)
 (-2.0, 2.0)
 (0.0, 1.0)
 (0.0, 1.0)

In [31]:
ystar

5-element Vector{Float64}:
 -0.8348591168958507
 -0.8029232710704508
 -0.7710057426082355
 -0.748970171360507
 -0.7508582077436516

In [33]:
xseed = generate_random_state(bounds,obs.config)

5-element Vector{Float64}:
  1.184674562359191
  1.3588659735988164
 -0.3938362697760245
  0.2655974667963825
  0.8603373368390377

In [57]:
@model function custom_posterior_model(bounds,logp̃_fcn::Function)
    x = Vector{Float64}(undef, length(bounds))
    for i in 1:length(bounds)
        x[i] ~ Uniform(bounds[i][1], bounds[i][2])
    end
    target = exp(logp̃_fcn(x))
    return target
end

custom_posterior_model (generic function with 6 methods)

In [58]:
t = 0
c = sample(
    custom_posterior_model(bounds,logp̃_fcn),
    NUTS(),
    1000,
)

Sampling   0%|█                                         |  ETA: N/A


Sampling 100%|██████████████████████████████████████████| Time: 0:00:11


TypeError: TypeError: in typeassert, expected Float64, got a value of type ForwardDiff.Dual{Nothing, Float64, 5}